In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from ripple_heterogeneity.utils import functions,loading,add_new_deep_sup
import seaborn as sns
from matplotlib.ticker import AutoMinorLocator

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [ ]:
# basepath = r"Z:\\Data\\AYAold\\AB1\\day1"
# loading.load_cell_metrics(basepath)


In [2]:
df_sessions = pd.read_csv(r'Z:\home\ryanh\projects\ripple_heterogeneity\sessions.csv')
df = loading.load_all_cell_metrics(df_sessions.basepath.unique())


In [16]:
df.query("brainRegion.str.contains('CA1')").groupby(["basepath","shankID"]).size().mean()

10.39423076923077

In [24]:
df.query("brainRegion.str.contains('CA1')").groupby(["basepath"]).size().max()


275

In [25]:
df.animal.unique(), df.animal.nunique()

(array(['AB1', 'AB3', 'AB4', 'AYA10', 'AYA4', 'AYA6', 'AYA7', 'AYA9',
        'OML22', 'OML23', 'EE', 'FF', 'GG', 'Rat08', 'Rat09', 'Rat10',
        'Rat11', 'Achilles', 'Buddy', 'Cicero', 'Gatsby', 'HMC1',
        '2006-4-10', '2006-4-18', '2006-6-12', '2006-6-13', '2006-6-7',
        'ec013', 'ec014', 'Kenji', 'ec016', 'f01_m', 'g01_m', 'i01_m',
        'j01_m', 'km01', 'nlx', 'OML10', 'OML18', 'OML19', 'OML3', 'OML5',
        'OML7', 'OML8', 'OR15', 'OR18', 'OR21', 'OR22', 'OR23'],
       dtype=object),
 49)

In [26]:
df.query("bad_unit==False & brainRegion.str.contains('CA1')").animal.unique(), df.query("bad_unit==False & brainRegion.str.contains('CA1')").animal.nunique()


(array(['AB1', 'AB3', 'AB4', 'AYA10', 'AYA4', 'AYA6', 'AYA7', 'AYA9',
        'OML22', 'OML23', 'EE', 'FF', 'GG', 'Rat08', 'Rat10', 'Rat11',
        'Achilles', 'Buddy', 'Cicero', 'Gatsby', 'HMC1', '2006-4-10',
        '2006-4-18', '2006-6-12', '2006-6-13', '2006-6-7', 'ec013',
        'ec014', 'Kenji', 'ec016', 'f01_m', 'g01_m', 'i01_m', 'km01',
        'nlx', 'OML10', 'OML18', 'OML19', 'OML3', 'OML5', 'OML7', 'OML8',
        'OR15', 'OR18', 'OR21', 'OR22', 'OR23'], dtype=object),
 47)

In [27]:
df = add_new_deep_sup.deep_sup_from_deepSuperficialDistance(df)

In [6]:
df.keys()[df.keys().str.contains("tag")]

Index(['peakVoltage', 'peakVoltage_expFit', 'tags_P', 'tags_N', 'tags_Bad',
       'tags_Pr', 'tags_Nr', 'tags_Pb', 'tags_Nb', 'tags_InverseSpike',
       'peakVoltage_expFitLengthConstant', 'tags_bad_waveform', 'tags_Noise',
       'tags_contaminated'],
      dtype='object')

In [14]:
df.animal.nunique()

49

In [29]:
df.query("deepSuperficialDistance.isna() & brainRegion.str.contains('CA1')")[["basepath","UID"]]

,basepath,UID
17465,Z:\Data\GrosmarkAD\Buddy\Buddy_06272013,19
17473,Z:\Data\GrosmarkAD\Buddy\Buddy_06272013,27
17789,Z:\Data\GrosmarkAD\Gatsby\Gatsby_08022013,25
17863,Z:\Data\GrosmarkAD\Gatsby\Gatsby_08282013,19
17868,Z:\Data\GrosmarkAD\Gatsby\Gatsby_08282013,24
17870,Z:\Data\GrosmarkAD\Gatsby\Gatsby_08282013,26
28643,Z:\Data\OMLproject\OML8\day17,19
28656,Z:\Data\OMLproject\OML8\day17,32


In [30]:
df.query("bad_unit==False & brainRegion.str.contains('CA1') & putativeCellType.str.contains('Pyr')").deepSuperficial.value_counts()


Deep           3658
middle         2706
Superficial    1946
Name: deepSuperficial, dtype: int64

In [31]:
df.query("bad_unit==False & brainRegion.str.contains('CA1') & putativeCellType.str.contains('Pyr')").deepSuperficial.value_counts().sum()

8310

## how many animals on each maze

In [66]:
epoch_df = pd.DataFrame()
for basepath in df.basepath.unique():
    epoch_df_temp = loading.load_epoch(basepath)
    epoch_df_temp["basepath"] = basepath
    epoch_df_temp["animal"] = loading.get_animal_id(basepath)
    epoch_df = pd.concat([epoch_df,epoch_df_temp],ignore_index=True)

In [67]:
epoch_df.environment.unique()

array(['sleep', 'linear', 'tmaze', 'cheeseboard', 'box', 'wmaze', 'water',
       'unknown', 'open', 'linearOne', 'linearTwo', 'Tmaze', 'wheel',
       'bigSquare', 'Mwheel', 'plus', 'midSquare', 'bigSquarePlus',
       'wheel_home', 'circle', 'ZigZag', 'test'], dtype=object)

In [69]:
epoch_df.query("environment=='box' | environment=='open' | environment=='bigSquare' | environment=='midSquare' | environment=='midSquare'").animal.nunique()


10

In [70]:
epoch_df.query("environment=='linear' | environment=='linearOne' | environment=='linearTwo'").animal.nunique()


27

In [57]:
epoch_df.query("environment=='tmaze' | environment=='Tmaze' | environment=='Mwheel'").animal.nunique()

15

In [74]:
epoch_df.query("environment=='cheeseboard'").animal.nunique()


9

In [59]:
epoch_df.query("environment=='wmaze'").animal.nunique()


6

In [71]:
epoch_df.query("environment=='bigSquarePlus' | environment=='plus'").animal.nunique()


1

In [72]:
epoch_df.query("environment=='ZigZag'").animal.nunique()


1